Les parties I et III ne comportent que de la programmation. 

La partie II ne comporte que des questions théoriques, **à rédiger sur une copie**.

La partie I est indépendante des deux autres. La seconde question de la partie III fait appel à la partie II.

Il n'est pas nécessaire de traiter l'ensemble du sujet pour obtenir une bonne note.

Penser à régulièrement enregistrer le notebook.

Le notebook devra être envoyé par e-mail à l'adresse joon.kwon@inrae.fr

# Partie I

On reprend le problème du plus grand nombre à 5 chiffres (cf. TD1, TP1, TP2), à ceci près que les chiffres sont ici tirées selon une distribution $\nu$ fixée mais inconnue, et que le choix d'un emplacement déjà pris est sanctionné par un paiement de -100000.

Le but de l'agent est de former un nombre à 5 chiffres, le plus grand possible. On considère 5 emplacements, initialement vides, correspondant aux chiffre des unités, des dizaines, des centaines, des milliers, et des dizaines de milliers. À chaque étape, un chiffre (entre 0 et 9) est tiré selon $\nu$ et présenté à l'agent. Celui-ci doit le placer dans l'un des emplacements disponibles. Le nombre est donc formé au bout de 5 étapes.

On modélise le problème par un MDP dont
- l'espace d'états est $\mathcal{S}=\left\{ 0,1 \right\}^5\times \left\{ 0,\dots,9 \right\}$ où pour les cinq premières composantes, un 1 correspond à un emplacement libre,
- l'ensemble de paiements est $\mathcal{R}=\left\{ 0,1,\dots,99999 \right\}$,
- l'ensemble d'actions est $\mathcal{A}=\left\{ 1,\dots,5 \right\}$,
- la dynamique de transition est 
$$p(\,\cdot\,|s,a)= \begin{cases} \delta_{-100000}\otimes \delta_s&\text{si $s^{(a)}=0$}\\ 
\delta_{10^{a-1}s^{(6)}} \otimes \delta_{\sigma(s,a)}\otimes
\nu&\text{si $s^{(a)}=1$}.
\end{cases}$$

On considère un paiement non-escompté ($\gamma=1$) et une distribution pour l'état initial $\mu=\delta_{(1,1,1,1,1)}\otimes \nu$.

L'implémentation de l'environnement ci-dessous reprend les conventions de la librairie Gymnasium. L'épisode est signalé comme terminé lorsque les 5 emplacements sont occupés (lorsque le nombre est formé) et comme tronqué au bout de 500 étapes.

In [ ]:
import random
import numpy as np

class nombre_cinq_chiffres():
    def __init__(self):
        np.random.seed(2025)
        x = np.random.uniform(low=0.0, high=2, size=10)
        exp_x = np.exp(x - np.max(x))
        self.d = exp_x / exp_x.sum()

        return None

    def reset(self):
        self.s = (1,1,1,1,1, np.random.choice(a=10, p=self.d))
        self.t = 0
        return self.s, None
    
    def step(self, a):
        number = list(self.s[:5])
        number[a-1] = 0

        if self.s[a-1] == 1:
            r = (10**(a-1))*self.s[5]
            self.s = (*number,np.random.choice(a=10, p=self.d))
        else:
            r = -100000

        terminated = (number == [0,0,0,0,0])

        self.t += 1
        truncated = (self.t == 500)

        return self.s, r, terminated, truncated, None

env =  nombre_cinq_chiffres()

# liste contenant tous les états
S = [(i1,i2,i3,i4,i5,i6) for i1 in [0,1] for i2 in [0,1] for i3 in [0,1] for i4 in [0,1] for i5 in [0,1] for i6 in range(10)]

# liste contenant tous les couples (état,action)
SA = [(s,a) for s in S for a in [1,2,3,4,5]]

**Question 1**: Implémenter un algorithme d'apprentissage par renforcement (pour le controle). Le faire tourner pendant (au moins) 10000 épisodes.

In [ ]:
gamma = 1
eps = .2
n_episodes = 10000

# initialisation 
q_cumul, n_updates = dict(), dict()
for sa in SA:
    q_cumul[sa] = 0.
    n_updates[sa] = 0

for episode in range(n_episodes):
    s = env.reset()[0]

    terminated, truncated = False, False

    while not (terminated or truncated):
        # compléter

**Question 2**: Afficher la politique $\pi$ obtenue.

**Question 3**: Estimer la quantité suivante en utilisant la politique obtenue $\pi$ sur 10000 épisodes.
$$\mathbb{E}_{\mu,\pi}\left[ \sum_{t=1}^{+\infty}R_t\right].$$

# Partie II (à rédiger sur une copie)

On considère le cadre et les notation du cours, avec un MDP donné.
Soit $\gamma\in (0,1)$ fixé et $\pi$ une politique stationnaire.

**Question 1**: Soit $N\geqslant 1$ et $\mu_1,\dots,\mu_N\geqslant 0$ tels que
$\sum_{n=1}^N\mu_n=1$.
On considère l'opérateur $\tilde{B}_{\pi}=\sum_{n=1}^N\mu_n(B_{\pi}^{(Q)})^n$ (où $(B_{\pi}^{(Q)})^n$ désigne
l'opérateur $B_{\pi}^{(Q)}$ itéré $n$ fois)

**a)** Montrer que $\tilde{B}_\pi$ est $\gamma$-lipschitzienne pour $\left\|\,\cdot\,\right\|_{\infty}$.

**b)** Déterminer les points fixes de $\tilde{B}_{\pi}$.

**Question 2**: Soit $0\leqslant \lambda<1$. On définit l'opérateur
$B_{\pi}^{\left[ \lambda \right] }$ par
$$B_{\pi}^{\left[ \lambda \right] }q=(1-\lambda)\sum_{n=0}^{+\infty}\lambda^{n-1}B_{\pi}^nq,\quad q\in \mathbb{R}^{\mathcal{S}\times \mathcal{A}}.$$

**a)** Montrer que $B_{\pi}^{\left[ \lambda \right] }$ est bien définie
et $\gamma$-lipschitzienne pour $\left\|\,\cdot\,\right\|_{\infty}$.

**b)**Déterminer les points fixes de cet opérateur.

**c)** Soit $q\in \mathbb{R}^{\mathcal{S}\times \mathcal{A}}$. Déterminer
la limite de $B_{\pi}^{\left[ \lambda \right] }q$ quand $\lambda\to 1^-$.

**Question 3**: Soit $q\in \mathbb{R}^{\mathcal{S}\times \mathcal{A}}$, $s\in
\mathcal{S}, a\in \mathcal{A}$ et 
$(S_0,A_0,R_1,S_1,A_1,R_2,\dots )$ une histoire infinie aléatoire
de loi $\mathbb{P}_{s,a,\pi}$.

**a)** Pour $n\geqslant 1$, rappeler l'estimateur stochastique de
$(B_{\pi}^nq)(s,a)$ vu en cours.

**b)** En déduire un estimateur stochastique pour $(B_{\pi}^{\left[ \lambda \right] }q)(s,a)$.

**Question 4**: Proposer un algorithme d'évaluation de politique (en programmation
dynamique) faisant appel à l'opérateur $B_{\pi}^{\left[ \lambda
\right] }$, et établir sa convergence.

**Question 5**: Proposer un algorithme de contrôle (en apprentissage par renforcement) faisant appel à l'estimateur obtenu à la question **3b** et à des politiques $\varepsilon$-gloutonnes.

**Question 6**: Soit $p\geqslant 1$ et $\left\{ q_w \right\}_{w\in \mathbb{R}^p}$ un sous-ensemble de $\mathbb{R}^{\mathcal{S}\times \mathcal{A}}$ telle que $w\mapsto q_w$ est différentiable. Proposer un algorithme d'apprentissage par renforcement de type semi-gradient faisant appel à la famille paramétrique $\left\{ q_w \right\}_{w\in \mathbb{R}^p}$, à l'estimateur de la question 3b, ainsi qu'à des politiques $\varepsilon$-gloutonnes.

# Partie III

Soit $p\geqslant 1$, $\left\{ q_w \right\}_{w\in \mathbb{R}^p}$ un sous-ensemble de $\mathbb{R}^{\mathcal{S}\times \mathcal{A}}$ telle que $w\mapsto q_w$ est différentiable, et $0\leqslant \lambda<1$. On définit l'algorithme de type semi-gradient suivant.

**Input**: number of episodes $N\geqslant 1$.

$w\leftarrow 0\in \mathbb{R}^p$

$z\leftarrow 0\in \mathbb{R}^p$

**for** $n=1,\dots,N$ **do**
> Observe $S_0$
>
> Choose $A_0$
>
> $t\leftarrow 0$  
> **while** episode is not terminated nor truncated **do**
>> Observe $R_{t+1},S_{t+1}$        
>> $A_{t+1}\sim \pi_{g,\varepsilon}\left[ q_w \right](S_{t+1}) $        
>> $\delta\leftarrow R_{t+1}+\gamma q_w(S_{t+1},A_{t+1})-q_w(S_t,A_t)$     
>> $z\leftarrow \gamma\lambda z+\nabla_wq_w(S_t,A_t)$        
>> Choose $\alpha$     
>> $w\leftarrow w+\alpha\delta z$        
>> $t\leftarrow t+1$

**Question 1**: En s'inspirant de la correction du TP6 (https://joon-kwon.github.io/rl-ups/TP6-correction.ipynb), implémenter l'algorithme ci-dessus pour résoudre le problème Mountain Car.

**Question 2**: Implémenter également l'algorithme obtenu à la question 6) de la Partie II et comparer.